In [1]:
from google.colab import drive
drive.mount('/content/drive')

IMG_PATH = '/content/drive/MyDrive/Practica_final/COVID-19-CT-Seg_20cases'
MASK_PATH = '/content/drive/MyDrive/Practica_final/Infection_Mask'


Mounted at /content/drive


In [2]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import layers, models
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [3]:
def load_nifti(filepath):
    return nib.load(filepath).get_fdata()

def preprocess_slice(img_slice, mask_slice, target_size=(128,128)):
    img_resized = cv2.resize(img_slice, target_size)
    mask_resized = cv2.resize(mask_slice, target_size, interpolation=cv2.INTER_NEAREST)

    # Normalización
    img_resized = (img_resized - np.min(img_resized)) / (np.max(img_resized) - np.min(img_resized) + 1e-7)

    return img_resized, mask_resized


In [5]:
X = []
Y = []

img_files = sorted([f for f in os.listdir(IMG_PATH) if f.endswith('.nii.gz')])

for file in img_files:
    img_file = os.path.join(IMG_PATH, file)
    mask_file = os.path.join(MASK_PATH, file)

    if not os.path.exists(mask_file):
        print(f"Máscara no encontrada para {file}, se omite.")
        continue

    img_vol = load_nifti(img_file)
    mask_vol = load_nifti(mask_file)

    for z in range(img_vol.shape[2]):
        img_slice = img_vol[:,:,z]
        mask_slice = mask_vol[:,:,z]

        if np.sum(mask_slice) > 50:
            img_prep, mask_prep = preprocess_slice(img_slice, mask_slice)
            X.append(img_prep)
            Y.append(mask_prep)

X = np.expand_dims(np.array(X), -1)
Y = np.expand_dims(np.array(Y), -1)
print(f"Slices cargados: {X.shape[0]}")


Slices cargados: 1837


In [6]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

print(f"Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")


Train: 1285, Val: 276, Test: 276
